# Boston Housing PCA und SVD

In [ ]:
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import StringIndexer, VectorAssembler, PCA
from pyspark.ml.classification import LinearSVC
from pyspark.sql.session import SparkSession, Row
from pyspark.sql.functions import desc, expr
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vectors
from helpers.helper_functions import translate_to_file_string

# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

In [ ]:
inputFile = translate_to_file_string("../data/Boston_Housing_Data.csv")

Spark session creation 

In [ ]:
spark = (SparkSession
       .builder
       .appName("BostonHousingPCA")
       .getOrCreate())

DataFrame creation using an ifered Schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) \
       .withColumn("CATBOOL", expr("CAT").cast(BooleanType()))
print(df.printSchema())

Prepare training and test data.

In [ ]:
featureCols = df.columns.copy() 
# TODO remove additional columns to test PCA
featureCols.remove("MEDV")
featureCols.remove("CAT")
featureCols.remove("CATBOOL") 
print(featureCols)

assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)
featureSet = assembler.transform(df)
# TODO test if standardization or normilization helps 

## PCA Model

In [ ]:
# TODO test different Settings and compare the results
pcaModel = PCA(inputCol="features",outputCol="pcaFeatures",k=7).fit(featureSet)
result = pcaModel.transform(featureSet).select("pcaFeatures")
printDf(result.limit(10))

## SVD

In [ ]:
featureVector = featureSet.select("features").rdd.map (lambda row : Vectors.dense(row))
mat = RowMatrix(featureVector)
# TODO test different settings and compare the result
svd = mat.computeSVD(7, True, 1.0e-9)
U = svd.U # The U factor is a RowMatrix.
s = svd.s # The singular values are stored in a local dense vector.
V = svd.V 
collectPartitions = U.rows.collect()
print("U factor is:")
for  vector in collectPartitions :
	   print("\t", vector)
print("Singular values are: ", s)
print("V factor is:\n",V )

In [ ]:
spark.stop()